# Using SQL for Business Decisions

We can use a database that stores records for song sales at a music store, Chinook, and use SQL to make business decisions based on different needs, such as evaluating what kinds of albums we should try to acquire more of, or which employees are performing well in terms of sales.

We begin by loading in the database to SQL, and exploring it.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

We use the above code to connect the project to our database (Chinook).

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Our data tracks the music store's purchases, including information on purchases, customers, and employees, but also on specific media like the song, artist, and album. This is all in the above tables.

As an example of the kinds of things we can do with the data, the following query creates two subqueries that build off of one another before executing the main query.

In [3]:
%%sql
WITH
    total_by_cust AS
    (
        SELECT i.customer_id cust, c.country, SUM(i.total) total_purchases
        FROM invoice i 
        INNER JOIN customer c on c.customer_id = i.customer_id
        GROUP BY 1, 2
        ),
   max_by_country AS
   (
       SELECT MAX(total_purchases) total_purchased, country, cust FROM total_by_cust
       GROUP BY 2
       )

SELECT mc.country country, 
    cu.first_name || " " || cu.last_name customer_name,
    mc.total_purchased total_purchased
    FROM max_by_country mc
    INNER JOIN customer cu ON cu.customer_id = mc.cust
    ORDER BY 1;
    

Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.38999999999999
Brazil,Luís Gonçalves,108.89999999999998
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02000000000001
Czech Republic,František Wichterlová,144.54000000000002
Denmark,Kara Nielsen,37.61999999999999
Finland,Terhi Hämäläinen,79.2


Specifically, this SQL code first creates a table of the total amount spent by each customer over all of that customer's purchases, including that customer's country. The next subquery takes only the max amount purchased from each country. The final main query simply reformats the data to include the customers' full name.

## Finding Best-Sellers under Specific Filters

Say we are told to find the genres that sell the most tracks within the United States. We need to filter our data to aggregate tracks sold, grouped by genre, and only including US sales. Let's write a query to do so.

In [4]:
%%sql
SELECT * FROM invoice_line
WHERE quantity != 1
LIMIT 10;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity


We can see the quantity for every track is always 1, so we don't have to account for this in our totals for our main queries, but we will do so anyway so the same query can be used if that ever changes.

In [5]:
%%sql
WITH
    total_rows AS
    (
        SELECT SUM(invoice_line.quantity) USA_rows
        FROM invoice_line
        INNER JOIN invoice ON invoice.invoice_id = invoice_line.invoice_id
        WHERE invoice.billing_country = "USA"
        )
SELECT gr.name AS Genre, SUM(il.quantity) AS Number_Sold, 
        ROUND((CAST(SUM(il.quantity) AS FLOAT)*100/(SELECT * FROM total_rows)),2) Percentage_Sold
    FROM invoice_line il
    INNER JOIN track tr ON tr.track_id = il.track_id
    INNER JOIN invoice inv ON inv.invoice_id = il.invoice_id
    INNER JOIN genre gr ON gr.genre_id = tr.genre_id
    WHERE inv.billing_country = "USA"
    GROUP BY 1
    ORDER BY 2 DESC;

Done.


Genre,Number_Sold,Percentage_Sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


We have taken the data grouped by genre, filtered only on USA data, and found that the most popular genres are Rock, Alternative/Punk, Metal, and R&B/Soul. So if we are trying to decide which kinds of albums to add to our store's shelves, we may choose a Rock or Punk album before a Pop or Hip-Hop album.

## Analyzing Employee Sales Performance

Say we want to look at each employee at our music store and evaluate how well they are selling. We can do this by looking at each customer's total purchases, and then adding up all of an employee's customers. 

In [6]:
%%sql
WITH
    total_by_cust AS (
        SELECT i.customer_id cust, SUM(i.total) total_purchases, c.support_rep_id support
            FROM invoice i 
            INNER JOIN customer c on c.customer_id = i.customer_id
            GROUP BY 1 )
SELECT  em.first_name || " " || em.last_name Employee, em.hire_date Hire_Date,
        ROUND(SUM(total_by_cust.total_purchases),2) Sales_Dollars
    FROM employee em
    INNER JOIN total_by_cust ON em.employee_id = total_by_cust.support
    GROUP BY 1

Done.


Employee,Hire_Date,Sales_Dollars
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


We can see that all three of our sales representatives are selling decent quantities. While Jane sells the most, she was hired first. Steve sold the least, but was hired six months after Jane. If anything, this makes it appear like Margaret is the least productive salesperson, as with only one month less than Jane, she is nearly 200 dollars behind her, but with five months more than Steve she is a only a similar amount ahead of him. It is all relative to the date this data is being collected though.

## Analyzing Sales by Country Using SQL Unions and With Clauses

What if we want to look at sales data for each country? We want a comprehensive list of countries, and for each one want to know:
* The total number of customers
* The total amount of sales
* The average value of sales per customer
* The average order value

We need to know this info for each country. So we first need to use a subquery to find the totals for each country, then another to format the data as desired, then a third with a union clause to separate out the countries with only a single customer into a compounded "Other" row, and then the main query sorts the data.

In [7]:
%%sql
WITH country_totals AS (
    SELECT cu.country, COUNT(DISTINCT cu.first_name) number_customers, 
        COUNT(inv.customer_id) number_orders, SUM(inv.total) sales 
    FROM invoice inv
    INNER JOIN customer cu ON cu.customer_id = inv.customer_id
    GROUP BY 1
    ),

country_finals AS (
    SELECT country, number_customers, ROUND(sales,2) total_sales, 
        ROUND(CAST(sales/number_customers AS FLOAT),2) sales_per_customer, 
        ROUND(CAST(sales/number_orders AS FLOAT),2) average_order_value,
        number_orders,
        CASE
            WHEN number_customers = 1 THEN 1
            ELSE 0
        END as more_or_less
        FROM country_totals
        ORDER BY number_customers
),

unsorted_final AS (
SELECT country, number_customers, total_sales, sales_per_customer, 
average_order_value, more_or_less
FROM country_finals
WHERE more_or_less = 0

UNION

SELECT "Other" country, SUM(number_customers) number_customers, 
    SUM(total_sales) AS total_sales, ROUND(AVG(sales_per_customer),2) AS sales_per_customer,
    ROUND(CAST(SUM(total_sales)/SUM(number_orders) AS FLOAT),2) average_order_value,
    more_or_less
FROM country_finals
WHERE more_or_less = 1
)
SELECT country, number_customers, total_sales, sales_per_customer, average_order_value
FROM unsorted_final
ORDER BY more_or_less, number_customers DESC

Done.


country,number_customers,total_sales,sales_per_customer,average_order_value
USA,12,1040.49,86.71,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.57,6.38
Other,15,1094.94,73.0,7.45


We can see now that while the most sales come from the United States, on a per customer and per order basis, the Czech Republic is the best bet. That said, the sample size is only two customers.

## Albums vs Indivdual Tracks

In an effort to save money, the store is considering purchasing only the most popular tracks from each album from record companies rather than all the tracks within an album.

We are to test if this is a viable strategy by finding what percentage of purchases are albums vs full tracks. Each purchase is either full albums or individual tracks, as that is how sales are divided at the store.

There are two edge cases:
* Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
* Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

We are going to handle the first by only counting albums with more than a few tracks. We will ignore the second as it does not happen often according to previous analysis.

In [8]:
%%sql
SELECT ln.*, tr.album_id
FROM invoice_line ln
INNER JOIN track tr ON tr.track_id = ln.track_id
LIMIT 10

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,album_id
1,1,1158,0.99,1,91
2,1,1159,0.99,1,91
3,1,1160,0.99,1,91
4,1,1161,0.99,1,91
5,1,1162,0.99,1,91
6,1,1163,0.99,1,91
7,1,1164,0.99,1,91
8,1,1165,0.99,1,91
9,1,1166,0.99,1,91
10,1,1167,0.99,1,91


In [12]:
%%sql
WITH album_distinct_tracks AS 
(
SELECT a.album_id album_id, 
        a.title title, 
        COUNT(DISTINCT(t.track_id)) album_track_num
FROM album a
INNER JOIN track t ON a.album_id = t.album_id
GROUP BY 1
),

album_purchase_invoice AS 
(
SELECT il.invoice_id invoice_id, 
        COUNT(DISTINCT(t.album_id)) invoice_album_count,
        COUNT(DISTINCT(t.track_id)) invoice_track_count,
        adt.album_track_num,
        t.album_id album_id,
        a.title album_title
FROM invoice_line il
INNER JOIN track t ON il.track_id = t.track_id
INNER JOIN album a ON t.album_id = a.album_id
INNER JOIN album_distinct_tracks adt ON adt.album_id = a.album_id
GROUP BY 1
HAVING ((invoice_album_count = 1) AND (invoice_track_count = adt.album_track_num))
),

total_invoice_count AS
(
SELECT COUNT(i.invoice_id)
FROM invoice i
),

album_invoice_count AS
(
SELECT COUNT(api.invoice_id)
FROM album_purchase_invoice api
),

non_album_invoice_count AS
(
SELECT album_invoice_count.*
FROM album_invoice_count
)

SELECT COUNT(api.invoice_id) album_invoices,
        CAST(COUNT(api.invoice_id) AS FLOAT)/(SELECT * FROM total_invoice_count) percentage_album_invoices,
        (SELECT * FROM total_invoice_count) - COUNT(api.invoice_id) non_album_invoices,
        (SELECT * FROM total_invoice_count) total_invoice_count,
        (((SELECT * FROM total_invoice_count) - COUNT(api.invoice_id))/CAST((SELECT * FROM total_invoice_count) AS FLOAT)) percentage_non_album_invoices
FROM album_purchase_invoice api

Done.


album_invoices,percentage_album_invoices,non_album_invoices,total_invoice_count,percentage_non_album_invoices
114,0.18566775244299674,500,614,0.8143322475570033


We first looked at how many different tracks were in each album. As we are ignoring the ege cases where customers purchase all tracks from an album and then some other individual tracks and instead only looking for purchases of singular full items, we then look in album_purchase_invoice for invoices with a single album, where there's only one distinct album ID and the same number of tracks as the full album. 

We then use some more subqueries each to get our final numbers.

Then we get our final table fully formatted, and see almost 19% of purchases are single albums. The Chinook store should likely continue to purchase full albums, as that is a significant amount of income.

### Conclusion

This project while a little all over the place, was able to show many applications of advanced SQL subqueries to make rational and informed business decisions.

Many of the complex queries here seem like they would be easier to construct by hand than by using SQL, but if the database had been mor extensie and realistic, that would no longer have been practical. 

Further analysis can be done on this daa, to find things like "which artists are used more often in playlists", or "how many tracks have been purchased in the past versus never purchased", but it would be more interesting with a larger data sample.